In [1]:
import pandas as pd
balancing_df = pd.read_csv("balancing_data.csv")
demand_df = pd.read_csv("demand_load_data.csv")
generation_df = pd.read_csv("generation_data.csv")
price_df = pd.read_csv("price_data.csv")

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plt.plot(balancing_df["System Price (ESO Outturn) - GB (£/MWh)"])
plt.title('Visual Inspection of Time Series')
plt.xlabel('Time')
plt.ylabel('Value')
plt.show()